In [1]:
%pip install newsapi-python


[notice] A new release of pip is available: 24.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='268353c215ad4a0d84c903ef353ed922')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(q='bitcoin', country='us')

# /v2/everything
all_articles = newsapi.get_everything(q='usa-immigration',
                                      from_param='2025-04-01',
                                      to='2025-04-18',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

# /v2/top-headlines/sources
sources = newsapi.get_sources()

In [17]:
all_articles['articles']

[{'source': {'id': 'the-jerusalem-post', 'name': 'The Jerusalem Post'},
  'author': 'BY MICHAEL STARR',
  'title': 'Columbia protest leader arrested by ICE at citizenship interview',
  'description': 'A US District Court Judge ruled Monday that Mahdawi not be removed from the United States or Vermont while the court proceeds with his petition against the arrest.',
  'url': 'https://www.jpost.com/breaking-news/article-850145',
  'urlToImage': 'https://images.jpost.com/image/upload/f_auto,fl_lossy/c_fill,g_faces:center,h_407,w_690/618113',
  'publishedAt': '2025-04-15T01:48:34Z',
  'content': 'Columbia University pro-Palestinian protest leader Mohsen Mahdawi was arrested by Department for Homeland Security agents on Monday in Colchester when attending a citizenship interview, according to … [+7428 chars]'},
 {'source': {'id': None, 'name': 'TheJournal.ie'},
  'author': 'Eoghan Dalton',
  'title': 'Tourism chief says Ireland should not rely on US tourists too much amid turmoil',
  'descri

In [22]:
all_articles

{'status': 'ok',
 'totalResults': 170,
 'articles': [{'source': {'id': 'the-jerusalem-post',
    'name': 'The Jerusalem Post'},
   'author': 'BY MICHAEL STARR',
   'title': 'Columbia protest leader arrested by ICE at citizenship interview',
   'description': 'A US District Court Judge ruled Monday that Mahdawi not be removed from the United States or Vermont while the court proceeds with his petition against the arrest.',
   'url': 'https://www.jpost.com/breaking-news/article-850145',
   'urlToImage': 'https://images.jpost.com/image/upload/f_auto,fl_lossy/c_fill,g_faces:center,h_407,w_690/618113',
   'publishedAt': '2025-04-15T01:48:34Z',
   'content': 'Columbia University pro-Palestinian protest leader Mohsen Mahdawi was arrested by Department for Homeland Security agents on Monday in Colchester when attending a citizenship interview, according to … [+7428 chars]'},
  {'source': {'id': None, 'name': 'TheJournal.ie'},
   'author': 'Eoghan Dalton',
   'title': 'Tourism chief says Irelan

In [26]:
len(all_articles['articles'])

66

In [21]:
response = all_articles.get("articles", [])
len(response)
response

[{'source': {'id': 'the-jerusalem-post', 'name': 'The Jerusalem Post'},
  'author': 'BY MICHAEL STARR',
  'title': 'Columbia protest leader arrested by ICE at citizenship interview',
  'description': 'A US District Court Judge ruled Monday that Mahdawi not be removed from the United States or Vermont while the court proceeds with his petition against the arrest.',
  'url': 'https://www.jpost.com/breaking-news/article-850145',
  'urlToImage': 'https://images.jpost.com/image/upload/f_auto,fl_lossy/c_fill,g_faces:center,h_407,w_690/618113',
  'publishedAt': '2025-04-15T01:48:34Z',
  'content': 'Columbia University pro-Palestinian protest leader Mohsen Mahdawi was arrested by Department for Homeland Security agents on Monday in Colchester when attending a citizenship interview, according to … [+7428 chars]'},
 {'source': {'id': None, 'name': 'TheJournal.ie'},
  'author': 'Eoghan Dalton',
  'title': 'Tourism chief says Ireland should not rely on US tourists too much amid turmoil',
  'descri

In [28]:
response[0]

{'source': {'id': 'the-jerusalem-post', 'name': 'The Jerusalem Post'},
 'author': 'BY MICHAEL STARR',
 'title': 'Columbia protest leader arrested by ICE at citizenship interview',
 'description': 'A US District Court Judge ruled Monday that Mahdawi not be removed from the United States or Vermont while the court proceeds with his petition against the arrest.',
 'url': 'https://www.jpost.com/breaking-news/article-850145',
 'urlToImage': 'https://images.jpost.com/image/upload/f_auto,fl_lossy/c_fill,g_faces:center,h_407,w_690/618113',
 'publishedAt': '2025-04-15T01:48:34Z',
 'content': 'Columbia University pro-Palestinian protest leader Mohsen Mahdawi was arrested by Department for Homeland Security agents on Monday in Colchester when attending a citizenship interview, according to … [+7428 chars]'}

In [29]:
response[0].get("title", "")

'Columbia protest leader arrested by ICE at citizenship interview'

In [30]:
response[0].get("description", "")

'A US District Court Judge ruled Monday that Mahdawi not be removed from the United States or Vermont while the court proceeds with his petition against the arrest.'

In [31]:
from langchain.docstore.document import Document

articles = []
text = f"{response[0].get('title', '')}\n{response[0].get('description', '')}\n{response[0].get('content', '')}"
source = response[0].get("url", "unknown")
articles.append(Document(page_content=text, metadata={"source": source}))

In [34]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(articles)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embeddings)

In [35]:
db